In [7]:
import pandas as pd
import plotly.express as px

In [8]:
input_path = "../../../results/road/survey.parquet"
output_path = "../../../results/road/freeflow/reference_survey.parquet"

start_hour, end_hour = 2, 5

In [9]:
# Read data
df_survey = pd.read_parquet(input_path)

In [10]:
# Prepare processing
df_survey["departure_hour"] = df_survey["departure_time"] // 3600
df_survey.loc[df_survey["departure_hour"] > 23, "departure_hour"] -= 24

df_plot = df_survey.groupby("departure_hour")["weight"].sum().reset_index()
df_plot["selected"] = df_plot["departure_hour"].between(start_hour, end_hour)
px.bar(df_plot, x = "departure_hour", y = "weight", color = "selected")

In [11]:
# Prepare reference data
df_reference = df_survey[df_survey["departure_hour"].between(start_hour, end_hour)]
df_reference = df_reference.rename(columns = { "survey_travel_time_s": "reference_travel_time_s" })

In [12]:
# Ignore unrealistic travel times
df_reference = df_reference[df_reference["reference_travel_time_s"] < 2.5 * 3600].copy()

In [13]:
df_reference[[
    "trip_id", "origin_x", "origin_y", "destination_x", "destination_y",
    "reference_travel_time_s", "weight"
]].to_parquet(output_path)